In [78]:
#@markdown #Imports
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

In [79]:
#@markdown ##Motor class
class DCMotor:
    def __init__(self, free_speed_rps, resistance_ohm, kT_Nm_A):
        self.free_speed_rps = free_speed_rps
        self.resistance_ohm = resistance_ohm
        self.kT_Nm_A = kT_Nm_A  # Torque constant
        self.kV_rpm_V = free_speed_rps / (12) # Speed constant (assuming 12V nominal voltage)
        self.inertia = 0.001 # Placeholder, needs to be calculated or measured.

    def simulate_step(self, voltage, current_limit, current_speed_rps, current_position_rot, TIME_STEP):
        # Calculate back EMF
        back_emf = current_speed_rps * 2 * np.pi / 60 / self.kV_rpm_V * 12

        # Calculate current (considering current limit)
        current = (voltage - back_emf) / self.resistance_ohm
        current = np.clip(current, -current_limit, current_limit)

        # Calculate torque
        torque = self.kT_Nm_A * current

        # Calculate angular acceleration
        angular_acceleration = torque / self.inertia

        # Update speed and position (using simple integration)
        new_speed_rps = current_speed_rps + angular_acceleration * TIME_STEP
        new_position_rot = current_position_rot + new_speed_rps * TIME_STEP

        return new_position_rot, new_speed_rps

In [80]:
#@markdown ##Create a motor

# Import necessary libraries
from IPython.display import display
import ipywidgets as widgets

# Initialize motor object with default Falcon 500 parameters
motor = DCMotor(free_speed_rps=120, resistance_ohm=0.039, kT_Nm_A=0.0192)

# Dropdown for motor type selection
motor_type = widgets.Dropdown(
    options=["Falcon 500", "Custom"],
    value="Falcon 500",
    description="Motor:"
)

# Dynamic widgets for motor parameters
free_speed_widget = widgets.FloatText(value=120, description="Free speed (RPS):")
resistance_widget = widgets.FloatText(value=0.039, description="Resistance (Ohm):")
kT_widget = widgets.FloatText(value=0.0192, description="kT (Nm A):")

# Update motor object whenever a parameter changes
def update_motor(change=None):
    global motor
    motor = DCMotor(
        free_speed_rps=free_speed_widget.value,
        resistance_ohm=resistance_widget.value,
        kT_Nm_A=kT_widget.value
    )

# Bind changes in widgets to the motor update function
free_speed_widget.observe(update_motor, names='value')
resistance_widget.observe(update_motor, names='value')
kT_widget.observe(update_motor, names='value')

# Update dynamic options based on motor type
def update_options(change):
    if change['new'] == "Falcon 500":
        free_speed_widget.value = 120
        resistance_widget.value = 0.039
        kT_widget.value = 0.0192
    elif change['new'] == "Custom":
        free_speed_widget.value = 0
        resistance_widget.value = 0
        kT_widget.value = 0

# Bind the dropdown to the update options function
motor_type.observe(update_options, names='value')

# Display widgets
display(motor_type, free_speed_widget, resistance_widget, kT_widget)


Dropdown(description='Motor:', options=('Falcon 500', 'Custom'), value='Falcon 500')

FloatText(value=120.0, description='Free speed (RPS):')

FloatText(value=0.039, description='Resistance (Ohm):')

FloatText(value=0.0192, description='kT (Nm A):')

In [93]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

TIME_STEP = 0.001  # 1ms time step
MAX_VOLTAGE = 12       # Max voltage
#@markdown ##PID prediction
duration = 1 #@param {type:"number"}
current_limit = 60

# PID Controller
def pid_controller(setpoint, kP, kI, kD, duration):
    time = np.arange(0, duration, TIME_STEP)
    position = 0
    speed = 0
    integral = 0
    previous_error = 0

    positions = []
    speeds = []
    voltages = []
    errors = []
    integrals = []
    derivatives = []

    for t in time:
        # Calculate error
        error = setpoint - position
        integral += error * TIME_STEP
        derivative = (error - previous_error) / TIME_STEP

        # PID Output
        voltage = kP * error + kI * integral + kD * derivative
        voltage = np.clip(voltage, -MAX_VOLTAGE, MAX_VOLTAGE)  # Clamp voltage

        # Apply voltage and simulate motor
        position, speed = motor.simulate_step(voltage, current_limit, speed, position, TIME_STEP)

        # Record values
        voltages.append(voltage)
        positions.append(position)
        speeds.append(speed)
        errors.append(error)
        integrals.append(integral)
        derivatives.append(derivative)
        previous_error = error

    return time, positions, speeds, voltages, errors, integrals, derivatives

# Visualization Function
def visualize_pid(setpoint, kP, kI, kD):
    global duration  # Simulation time in seconds
    time, positions, speeds, voltages, errors, integrals, derivatives = pid_controller(setpoint, kP, kI, kD, duration)

    # Plot Results
    plt.figure(figsize=(20, 15))

    # Position
    plt.subplot(3, 2, 1)
    plt.plot(time, positions, label="Position (Rotations)")
    plt.axhline(setpoint, color="r", linestyle="--", label="Setpoint")
    plt.title("Position vs Time")
    plt.xlabel("Time (s)")
    plt.ylabel("Position (Rotations)")
    plt.legend()

    # Speed
    plt.subplot(3, 2, 2)
    plt.plot(time, speeds, label="Speed (RPS)", color="orange")
    plt.title("Speed vs Time")
    plt.xlabel("Time (s)")
    plt.ylabel("Speed (RPS)")
    plt.legend()

    # Voltage
    plt.subplot(3, 2, 3)
    plt.plot(time, voltages, label="Voltage (V)", color="green")
    plt.title("Voltage vs Time")
    plt.xlabel("Time (s)")
    plt.ylabel("Voltage (V)")
    plt.legend()

    # Error
    plt.subplot(3, 2, 4)
    plt.plot(time, errors, label="Error", color="purple")
    plt.title("Error vs Time")
    plt.xlabel("Time (s)")
    plt.ylabel("Error")
    plt.legend()

    # Error Integral
    plt.subplot(3, 2, 5)
    plt.plot(time, integrals, label="Error Integral", color="blue")
    plt.title("Error Integral vs Time")
    plt.xlabel("Time (s)")
    plt.ylabel("Error Integral")
    plt.legend()

    # Error Derivative
    derivatives[0] = 0  # First derivative is undefined

    plt.subplot(3, 2, 6)
    plt.plot(time, derivatives, label="Error Derivative", color="red")
    plt.title("Error Derivative vs Time")
    plt.xlabel("Time (s)")
    plt.ylabel("Error Derivative")
    plt.legend()

    plt.show()

# Interactive Widgets
interact(
    visualize_pid,
    setpoint=FloatSlider(value=3, min=0, max=20, step=1, description="Setpoint"),
    kP=FloatSlider(value=14, min=0, max=10, step=0.01, description="kP"),
    kI=FloatSlider(value=0.0, min=0, max=10, step=0.01, description="kI"),
    kD=FloatSlider(value=0.0, min=0, max=10, step=0.01, description="kD")
)

interactive(children=(FloatSlider(value=3.0, description='Setpoint', max=20.0, step=1.0), FloatSlider(value=10…

<function __main__.visualize_pid(setpoint, kP, kI, kD)>